ыыыыы

In [219]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
import joblib


df_old = pd.read_csv("marketing.csv")

selected_cols = [
    'Age', 'Gender', 'CampaignChannel',
    'AdSpend', 'ClickThroughRate','WebsiteVisits',
    'TimeOnSite', 'Conversion'
]


df_new = df_old[selected_cols]



df_new.head(10)



,Age,Gender,CampaignChannel,AdSpend,ClickThroughRate,WebsiteVisits,TimeOnSite,Conversion
0,56,Female,Social Media,6497.870068,0.043919,0,7.396803,1
1,69,Male,Email,3898.668606,0.155725,42,5.352549,1
2,46,Female,PPC,1546.429596,0.277490,2,13.794901,1
3,32,Female,PPC,539.525936,0.137611,47,14.688363,1
4,60,Female,PPC,1678.043573,0.252851,0,13.993370,1
5,25,Female,Social Media,9579.388247,0.153795,6,7.752831,1
6,38,Female,Referral,7302.899852,0.040975,42,10.698672,1
7,56,Female,Social Media,5324.283667,0.052878,48,2.987817,1
8,36,Female,Email,9421.250951,0.023536,13,14.287421,1
9,40,Male,Social Media,6229.193333,0.066641,22,4.613312,1


In [220]:
# One-Hot Encoding
df = pd.get_dummies(df_new, columns=['Gender', 'CampaignChannel'], drop_first=True)

# Проверим названия колонок
print(df.columns)
print(df.head())


Index(['Age', 'AdSpend', 'ClickThroughRate', 'WebsiteVisits', 'TimeOnSite',
       'Conversion', 'Gender_Male', 'CampaignChannel_PPC',
       'CampaignChannel_Referral', 'CampaignChannel_SEO',
       'CampaignChannel_Social Media'],
      dtype='object')
   Age      AdSpend  ClickThroughRate  WebsiteVisits  TimeOnSite  Conversion  \
0   56  6497.870068          0.043919              0    7.396803           1   
1   69  3898.668606          0.155725             42    5.352549           1   
2   46  1546.429596          0.277490              2   13.794901           1   
3   32   539.525936          0.137611             47   14.688363           1   
4   60  1678.043573          0.252851              0   13.993370           1   

   Gender_Male  CampaignChannel_PPC  CampaignChannel_Referral  \
0        False                False                     False   
1         True                False                     False   
2        False                 True                     False   
3   

In [221]:
data = df.copy()

# Выбираем колонки для скейлинга
scale_cols = ['Age', 'AdSpend']

# Создаём scaler и обучаем на этих колонках
scaler = StandardScaler()
data[scale_cols] = scaler.fit_transform(data[scale_cols])

# Проверим результат
data.head(10)




,Age,AdSpend,ClickThroughRate,WebsiteVisits,TimeOnSite,Conversion,Gender_Male,CampaignChannel_PPC,CampaignChannel_Referral,CampaignChannel_SEO,CampaignChannel_Social Media
0,0.830400,0.527484,0.043919,0,7.396803,1,False,False,False,False,True
1,1.702775,-0.388418,0.155725,42,5.352549,1,True,False,False,False,False
2,0.159343,-1.217296,0.277490,2,13.794901,1,False,True,False,False,False
3,-0.780138,-1.572106,0.137611,47,14.688363,1,False,True,False,False,False
4,1.098823,-1.170918,0.252851,0,13.993370,1,False,True,False,False,False
5,-1.249878,1.613343,0.153795,6,7.752831,1,False,False,False,False,True
6,-0.377503,0.811159,0.040975,42,10.698672,1,False,False,True,False,False
7,0.830400,0.113938,0.052878,48,2.987817,1,False,False,False,False,True
8,-0.511715,1.557619,0.023536,13,14.287421,1,False,False,False,False,False
9,-0.243292,0.432808,0.066641,22,4.613312,1,True,False,False,False,True


In [ ]:
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import joblib
import numpy as np

# --- 1. Логарифмируем CTR для сглаживания ---
data['CTR_log'] = np.log1p(data['ClickThroughRate'])

# --- 2. Разделяем X и y ---
X = data.drop('Conversion', axis=1)
y = data['Conversion']

# --- 3. Train/test split ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# --- 4. XGBoost без калибровки ---
xgb_model = XGBClassifier(
    random_state=42,
    n_estimators=300,
    learning_rate=0.07,    # немного быстрее
    max_depth=4,           # чуть глубже
    min_child_weight=4,    # меньше ограничение для веток
    reg_lambda=5,          # умеренная регуляризация
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='logloss'
)

# --- 5. Пайплайн ---
pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('model', xgb_model)
])

# --- 6. Обучаем ---
pipeline.fit(X_train, y_train)

# --- 7. Сохраняем модель ---
joblib.dump(pipeline, 'xgboost_no_calibration.pkl')
print("✅ Модель XGBoost без калибровки сохранена!")


✅ Модель с IsActive и Engagement сохранена!
